In [1]:
import requests
from bs4 import BeautifulSoup

kjarkas_id = 8988
saviaandina_id = 37055
calamarka_id = 23997
angelesazules_id = 16643
mana_id = 9300



def extract_lyrics_links(artist_url: str) -> list[str]:
    response = requests.get(url=artist_url)
    print(f"Response from artist page: {response.status_code}")
    soup = BeautifulSoup(response.text, "html.parser")
    links_ul = soup.find("ul", class_="listado-letras")
    links = [a["href"] for a in links_ul.find_all("a")]
    return links

def extract_song_lyric(song_url: str) -> str:
    response = requests.get(url=song_url)
    response.encoding = "utf-8"
    print(f"Response from lyrics page: {response.status_code}")
    soup = BeautifulSoup(response.text, "html.parser")
    song_title = soup.find("h1").get_text()
    print(f"Title: {song_title}")
    header = soup.find("h2", string="LETRA")
    if header is None:
        header = soup.find("h2", string="LETRA EN ESPAÑOL")
        
    if header is None:
        return ""

    paragraphs = []

    for p in header.find_all_next("p"):
        if p.find_parent("div") is None:
            continue
        if p.find_parent("div").get("id") == "letra":
            paragraphs.append(p.get_text(separator="\n"))

    lyrics = "\n".join(p for p in paragraphs)
    return f"Título: {song_title} \n\n {lyrics}"

def extract_artist_lyrics(artist: str, artist_id: int):
    lyrics_url = f"https://www.musica.com/letras.asp?letras={artist_id}&orden=alf"
    links = extract_lyrics_links(lyrics_url)
    print(f"found {len(links)} songs")
    artist_header = f"{artist}\n===\n"
    lyrics = []
    for link in links:
        print(f"extracting song from: {link}")
        lyrics.append(extract_song_lyric(link))
    lyrics_str = "\n\n===\n\n".join(lyrics)
    artist_str = artist_header + lyrics_str
    with open(f"{artist_id}_{artist}.txt", "w", encoding="utf-8") as file:
        file.write(artist_str)
    return artist_str

In [2]:
extract_artist_lyrics("Kjarkas", kjarkas_id)
extract_artist_lyrics("Savia Andina", saviaandina_id)
extract_artist_lyrics("Calamarka", calamarka_id)
extract_artist_lyrics("Angeles Azules", angelesazules_id)
extract_artist_lyrics("Mana", mana_id)

Response from artist page: 200
found 166 songs
extracting song from: https://www.musica.com/letras.asp?letra=1645995
Response from lyrics page: 200
Title: A las orillas del rio
extracting song from: https://www.musica.com/letras.asp?letra=1567692
Response from lyrics page: 200
Title: A los 500 años
extracting song from: https://www.musica.com/letras.asp?letra=2066364
Response from lyrics page: 200
Title: A nadie
extracting song from: https://www.musica.com/letras.asp?letra=989973
Response from lyrics page: 200
Title: A que volviste
extracting song from: https://www.musica.com/letras.asp?letra=1645999
Response from lyrics page: 200
Title: A tu ventana
extracting song from: https://www.musica.com/letras.asp?letra=2108958
Response from lyrics page: 200
Title: Acto Final
extracting song from: https://www.musica.com/letras.asp?letra=6580
Response from lyrics page: 200
Title: Al Final
extracting song from: https://www.musica.com/letras.asp?letra=989976
Response from lyrics page: 200
Title: A

'Mana\n===\nTítulo: Adicto A Tu Amor \n\n Adicto A Tu Amor\nCon ella\nEstoy adicto a su sexy amor\nPara mi en su velar, me hace alucinar\nSoy un convicto de tu amor\nY es que te beso en la cocina\nTe hago el amor en la oficina\nNo logro evitar, voy a penetrar\nEse arcoiris me fascina\nSuben, mis labios quieren subir hasta tus caderas\nMis labios quieren bajar, hasta tu pecera\nMami soy un adicto de tu amor, no lo puedo negar\nEres mi pasión, eres mi obsesión\nEres tu mi adicción, eres tu mi sol\nSoy adicto de tu amor, no lo puedo negar\nSoy adicto de tu amor\nEres mi pasión, eres mi obsesión\nEres tu mi adicción, eres tu mi sol\nAddict, no lo puedo negar\nAddict...\nQue manera hermosa como muerdes\nHay algo de sexy en tu mirar\nSoy como un volcán y tu falda cae\nEstamos empapados en amor\nSuben, mis labios quieren subir hasta tus caderas\nMis labios quieren bajar, hasta tu pecera\nMami soy un adicto de tu amor, no lo puedo negar\nEres mi pasión, eres mi obsesión\nEres tu mi adicción, e

In [23]:
from dotenv import load_dotenv
import openai
import os
from llama_index.core import (
    VectorStoreIndex, 
    SimpleDirectoryReader, 
    StorageContext,
    load_index_from_storage,
    Document,
    PromptTemplate
)
load_dotenv()
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI

# override default LLM 
llm = OpenAI(model="gpt-4o-mini")
Settings.llm = llm


In [24]:
def get_artist_documents(filename: str) -> list[Document]:
    with open(filename) as file:
        data = file.read()
    songs = data.split("===")
    artist = songs.pop(0).strip()
    
    documents = [
        Document(
            text=song,
            metadata={
                "category":"music",
                "artist": artist,
            }
        )
        for song in songs
    ]    
    return documents
    

In [25]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base")
Settings.embed_model = embed_model

In [26]:
PERSIST_DIR = "lyrics_index"

if not os.path.exists(PERSIST_DIR):
    documents = get_artist_documents("8988_Kjarkas.txt")
    index = VectorStoreIndex.from_documents(documents, show_progress=True)
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR) 
    index = load_index_from_storage(storage_context)

In [27]:
query_engine = index.as_query_engine(verbose=True)

In [28]:
qa_template_str = """
    You are an expert in Bolivian Folk music, your task is to guide and teach the user 
    about your field. Answer the user queries only with supported data in your context.
    Your context may contain complete lyrics or parts of them in different languages, but
    your answer will always be in Spanish. 

    Context information is below.
    ---------------------
    {context_str}
    ---------------------
    Given the context information and not prior knowledge, 
    answer the query with detailed source information, include direct quotes and use bullet lists in your 
    answers, in one of the bullets detail the tone/sentiment of the song.
    Query: {query_str}
    Answer: 
"""
qa_template = PromptTemplate(qa_template_str)

In [29]:
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

In [30]:
response = query_engine.query("cuales canciones de los Kjarkas hablan de abandono?")

In [31]:
from IPython.display import Markdown, display
display(Markdown(response.response))

Las canciones de los Kjarkas que hablan de abandono son:

- **"Llorando se fue"**
  - Esta canción trata sobre el dolor de una separación y el recuerdo de un amor perdido. 
  - Frases clave:
    - "Llorando se fue y me dejo solo y sin su amor"
    - "Sola estará recordando este amor que el tiempo no puede borrar"
  - **Sentimiento**: La canción transmite un tono de tristeza y melancolía por la pérdida de un amor.

- **"El adios"**
  - En esta canción se aborda el tema de la despedida y el reconocimiento de que el amor no siempre es correspondido.
  - Frases clave:
    - "adios mi amor buena suerte hasta que el tiempo nos junte"
    - "perdoname si al final solo te causado mal"
  - **Sentimiento**: La canción tiene un tono de resignación y aceptación, reflejando el dolor de la separación pero también un deseo de bienestar para la otra persona.

Ambas canciones reflejan el tema del abandono y el dolor que este provoca en las relaciones amorosas.

In [32]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='19962fee-3e2c-4030-9c52-e18a03c6bf06', embedding=None, metadata={'category': 'music', 'artist': 'Kjarkas'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3a1b3116-648a-4810-94c9-dac1e1d3d045', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'category': 'music', 'artist': 'Kjarkas'}, hash='73db528cf327371bab0c26d5a454eedb385dc9c18e751056e7e115a1632b1f94')}, text='Título: Llorando se fue \n\n Llorando se fue\nLlorando se fue\ny me dejo solo y sin su amor\nLlorando se fue\ny me dejo solo y con dolor\nSola estará recordando este amor\nque el tiempo no puede borrar\nSola estará recordando este amor\nque el tiempo no puede borrar\nLa recuerdo hoy\ny en mi pecho no existe el rencor\nLa recuerdo hoy\ny en mi pecho solo existe amor\nLlorando estará recordando este amor\nque un dia no supo cuidar\nLlorando estará recordando este amor\nque un dia no supo cuidar\n(Bis)', star

In [33]:
chat_engine = index.as_chat_engine(verbose=True)

In [34]:
response = chat_engine.chat("que canciones de los kjarkas hablan de abandono?")

Added user message to memory: que canciones de los kjarkas hablan de abandono?
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"canciones de los Kjarkas que hablan de abandono"}
Got output: Las canciones de Kjarkas que hablan de abandono son "Al partir" y "El adios". Ambas canciones expresan sentimientos de tristeza y despedida, reflejando el dolor de dejar a un ser querido.



In [36]:
display(Markdown(response.response))

Las canciones de Los Kjarkas que hablan de abandono son "Al partir" y "El adiós". Ambas expresan sentimientos de tristeza y despedida, reflejando el dolor de dejar a un ser querido.

In [37]:
display(Markdown(chat_engine.chat("y cuales hablan de la naturaleza?").response))

Added user message to memory: y cuales hablan de la naturaleza?
=== Calling Function ===
Calling function: query_engine_tool with args: {"input":"canciones de los Kjarkas que hablan de la naturaleza"}
Got output: Las canciones de Kjarkas que mencionan elementos de la naturaleza incluyen "Canto a la madre" y "Canto a la mujer de mi pueblo." Ambas canciones hacen referencia a montañas, pueblos perdidos, y la conexión con la tierra, destacando la belleza y la esperanza que se encuentran en el entorno natural.



Las canciones de Los Kjarkas que hablan de la naturaleza incluyen "Canto a la madre" y "Canto a la mujer de mi pueblo." Ambas hacen referencia a montañas, pueblos perdidos y la conexión con la tierra, destacando la belleza y la esperanza que se encuentran en el entorno natural.